# Fine-tuned YAMNet: small plane detection

In [ ]:
import pyaudio, librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Decide what type of messages are displayed by TensorFlow (ERROR, WARN, INFO, DEBUG, FATAL)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
## TensorFlow memory allocation options:
# # OPTION 1: "smart" allocation
config=tf.ConfigProto()
config.gpu_options.allow_growth=True
sess=tf.Session(config=config) 

# OPTION 2: maximum memory allocation per session (0-1 = 0-100%)
# gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
# config = tf.ConfigProto(gpu_options=gpu_options)
# sess=tf.Session(config=config)

# OPTION 3: ???
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

# OPTION 4: ???
#Slim/embedded versions: (https://github.com/google-research/tf-slim)

print("tf version: ", tf.__version__)
print("tf.keras version: ", tf.keras.__version__)

In [ ]:
## Add/append required paths
import os, sys

path_root = '/home/ups/Proyectos/Vigia_sonido/Models/yamnet_planes/' #path to main folder
# path_root = input("Enter the path of your repository: ") # ask user for path_root
assert os.path.exists(path_root)
sys.path.append(path_root)

path_yamnet_original = path_root+'yamnet_original/' #path to original yamnet files
assert os.path.exists(path_yamnet_original)
# sys.path.append(path_yamnet_original)

In [ ]:
## YAMNet (original) is only used to extract class names
import yamnet_original.yamnet as yamnet_original

class_names = yamnet_original.class_names(path_yamnet_original+'yamnet_class_map.csv')

In [ ]:
## Modified YAMNet model for feature extraction
import yamnet_modified as yamnet_modified
import yamnet_original.params as params

params.PATCH_HOP_SECONDS = 0.96 #low values: higher accuracy but higher computational cost

yamnet_features = yamnet_modified.yamnet_frames_model(params)
yamnet_features.load_weights(path_root+'yamnet.h5')

## YAMNet (original) predictions on a single audio

In [ ]:
#Load functions
%run "yamnet_functions.py"

In [ ]:
# Understand impact of parameters on input image and visualise
import yamnet_original.features as features

DESIRED_SR = params.SAMPLE_RATE # required by YAMNet

# Load waveform
file_name = "/home/ups/Proyectos/Vigia_sonido/Datasets/airplanes_v0/training_data/plane/track_04_0000_norm.wav"
waveform = read_wav(file_name, DESIRED_SR, use_rosa=True)
DESIRED_DURATION = 4.975 #[s]
waveform = waveform[0:round(DESIRED_DURATION*DESIRED_SR)] #the last data point does not need to be included in the STFT

print('Waveform size:', waveform.shape, 'and duration:', waveform.shape[0]/DESIRED_SR, '[s]\n')

# Calculate spectrogram
_, spectrogram = features.waveform_to_log_mel_spectrogram(waveform,params,print_on=1)
spectrogram = spectrogram.eval(session=sess)

print('Spectrogram size (x,y):',spectrogram.shape)

# Calculate log-mel spectrogram and scores
log_mel_spectrogram, _, _, scores = yamnet_features.predict(np.reshape(waveform, [1, -1]), steps=1)

print('Log-mel spectrogram size (x,y):',log_mel_spectrogram.shape)
print('Scores size (frames,classes):',scores.shape)

In [ ]:
# Visualize the results
plt.figure(figsize=(10, 10))

# Plot the waveform
plt.subplot(4, 1, 1)
plt.plot(waveform)
plt.xlim([0, len(waveform)])

# Plot the spectrogram
plt.subplot(4, 1, 2)
plt.imshow(spectrogram.T, aspect='auto', interpolation='nearest', origin='bottom')

# Plot the log-mel spectrogram (returned by the model)
plt.subplot(4, 1, 3)
plt.imshow(log_mel_spectrogram.T, aspect='auto', interpolation='nearest', origin='bottom')

# Plot and label the model output scores for the top-scoring classes
mean_scores = np.mean(scores, axis=0)
top_N = 10
top_class_indices = np.argsort(mean_scores)[::-1][:top_N]
plt.subplot(4, 1, 4)
plt.imshow(scores[:, top_class_indices].T, aspect='auto', interpolation='nearest', cmap='gray_r')
# Compensate for the PATCH_WINDOW_SECONDS (0.96 s) context window to align with log-mel spectrogram.
patch_padding = (params.PATCH_WINDOW_SECONDS / 2) / params.PATCH_HOP_SECONDS
plt.xlim([-patch_padding, scores.shape[0] + patch_padding])
# Label the top_N classes.
yticks = range(0, top_N, 1)
plt.yticks(yticks, [class_names[top_class_indices[x]] for x in yticks])
_ = plt.ylim(-0.5 + np.array([top_N, 0]))

plt.show()

## Load training data


In [ ]:
samples, labels = load_data(
    "/home/ups/Proyectos/Vigia_sonido/Datasets/airplanes_v0/training_data/",
    yamnet_features,
    num_augmentations=1, 
    max_sample_seconds=5.0, 
    DESIRED_SR=DESIRED_SR)

In [ ]:
import random

idxs = list(range(len(labels)))
random.shuffle(idxs)

samples = [samples[i] for i in idxs]
labels = [labels[i] for i in idxs]

samples = np.array(samples)
labels = np.array(labels)

print(" Loaded samples: " , samples.shape, samples.dtype,  labels.shape)


## Classifier which takes YAMNet's feature vector as input

In [ ]:
from tensorflow.keras import Model, layers

def yamnet_classifier(input_size=1024,
    num_hidden=1024,
    num_classes=2):

    input_layer = layers.Input(shape=(input_size,))    
    dense_layer = layers.Dense(num_hidden, activation=None)(input_layer)
    classifier_layer = layers.Dense(num_classes, activation='softmax')(dense_layer)
    model = Model(inputs=input_layer, outputs=classifier_layer)

    return model


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
save_best = ModelCheckpoint('top_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
from tensorflow.keras.optimizers import SGD, Adam

# Classifier definition
feature_vector_length = 1024
num_hidden = 1024
NUM_CLASSES = 2

yamnet_planes = yamnet_classifier(input_size=feature_vector_length, 
                      num_hidden=num_hidden,
                      num_classes=NUM_CLASSES)

#opt = Adam(learning_rate=0.001)
opt = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

yamnet_planes.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model 
history = yamnet_planes.fit(samples, labels, epochs=200, validation_split=0.1)
# history = yamnet_planes.fit(samples, labels, epochs=200, validation_split=0.1, callbacks=[save_best])

In [ ]:
# Load model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam

yamnet_planes = load_model(path_root+'top_model_v2.h5')
opt = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
yamnet_planes.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
yamnet_planes.summary()

## Test on holdout sounds

In [ ]:
## Specify class labels
class_labels = ["not plane", "plane"]

In [ ]:
# Scores for a single holdout audio
file_name = "/home/ups/Proyectos/Vigia_sonido/Datasets/airplanes_v2/holdout_data/plane/track_04_0000_norm.wav"
waveform = read_wav(file_name, DESIRED_SR, use_rosa=1)

scores = run_models(waveform, yamnet_features, yamnet_planes)
winner = class_labels[scores.argmax()]

print(" Best score: {}  label: {}".format(scores.max(), winner))

In [ ]:
# Scores for a holdout folder
holdout_dir = "/home/ups/Proyectos/Vigia_sonido/Datasets/airplanes_v0/holdout_data/not_plane/"
arr = os.listdir(holdout_dir)

for fname in arr: 
    fname = holdout_dir+fname
    waveform = read_wav(fname, DESIRED_SR, use_rosa=1)

    # make file a bit longer by duplicating it 
    waveform = np.concatenate((waveform,waveform,waveform))
    scores = run_models(waveform, yamnet_features, yamnet_planes, strip_silence=False)
    winner_save = np.empty((0,2))
    if scores[0] != -1:
        winner = class_labels[scores.argmax()]
        print(" Best score: {}  label: {}".format(scores.max(), winner))
        # winner_save = np.append(winner_save,np.array([scores.max(),winner]),axis=0)

In [ ]:
# result_array = np.empty((0, 2))
# result = np.array([1,"a"])
# result_array = np.append(result_array, [result], axis=0)
# result_array = np.append(result_array, [result], axis=0)

# result_array

In [ ]:
## Save the model for later
model.save("top_model_v2.h5", include_optimizer=False)